In [2]:
%run "./5. CNN Definition.ipynb"

X: (800, 360, 25)
Y: (800,)


In [3]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable

In [4]:
def calculate_class_activation_map(model, input_tensor, target_class):
    # Set the model into evaluation mode
    model.eval()

    logits = model(input_tensor)
    probabilities = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    # Check if the predicted class matches the target class for interpretation
    if predicted_class == target_class:
        # Obtain the gradients of the target class with respect to the output scores
        one_hot_output = torch.zeros_like(logits)
        one_hot_output[0, target_class] = 1
        logits.backward(gradient=one_hot_output)

        # Get the gradients of the first convolutional layer
        gradients = model.conv1d.weight.grad.squeeze().cpu().data.numpy()

        # Compute the channel importance weights by taking the mean across time steps
        channel_weights = np.mean(gradients, axis=-1)

        # Normalize the weights to [0, 1]
        channel_weights = (channel_weights - np.min(channel_weights)) / (np.max(channel_weights) - np.min(channel_weights))

        return channel_weights
    else:
        return None


In [5]:
model = torch.load("./models/model_1.pth")
test_indices = np.load("./models/model_1_test.npy")
input = np.array([X[test_indices][0]])
label = Y[test_indices][0]

input_data = Variable(torch.from_numpy(input).float())

cam = calculate_class_activation_map(model, input_data, target_class = label)

print(cam.shape)

(360, 360)
